In [1]:
# %matplotlib inline
import matplotlib as plt
plt.use('pdf')
import networkx as nx
from tqdm import tqdm
from matplotlib import pyplot as plt

from graph_generator import grid_2d, add_p_and_delta
from utils import infeciton_time2weight
from plot_utils import plot_snapshot
from ic import make_partial_cascade, sample_graph_from_infection
from collections import defaultdict
from core import normalize_mu
from synthetic_data import load_data_by_gtype
from build_edge_reward_table import build_reward_table
from edge_mwu import reward_by_infection_direction, reward_by_uninfected_node

In [2]:
if True:
    p = 0.7
    g = grid_2d(10)
    g = add_p_and_delta(g, p, 1)
    pos = {n: np.array(n) for n in g.nodes()}
else:
    g = load_data_by_gtype('kr-hier', '10-10')[0]

In [3]:
tbl1, tbl2 = build_reward_table(g)

100%|██████████| 100/100 [00:02<00:00, 35.03it/s]


In [4]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
q = (4, 4)
u = (4, 5)
reward = reward_by_infection_direction(g, q, u)
plot_snapshot(g, pos, reward, query_node=q, queried_nodes={u}, ax=ax[0])
ax[1].hist(list(reward.values()))
ax[0].set_title('reward value each expert/node')
ax[1].set_title('reward distribution')
ax[1].set_xlabel('reward value')
ax[1].set_ylabel('probability')
fig.savefig('figs/edge-reward-demo/direction.pdf')

In [5]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
reward = reward_by_uninfected_node(g, q)
plot_snapshot(g, pos, reward, query_node=q, ax=ax[0])
ax[1].hist(list(reward.values()))
ax[0].set_title('reward value each expert/node')
ax[1].set_title('reward distribution')
ax[1].set_xlabel('reward value')
ax[1].set_ylabel('probability')
fig.savefig('figs/edge-reward-demo/uninfected.pdf')

In [ ]:
def mwu_by_infection_direction(g, obs_nodes, infection_times, source,
                               direction_reward_table=None,
                               inf_reward_table=None,
                               save_logs=False,
                               debug=False):
    mu = {n: 1 for n in g.nodes_iter()}
    sp_len = nx.shortest_path_length(g, weight='d')
    centroids = []
    queried_nodes = set(obs_nodes)
    while True:
        if len(queried_nodes) == g.number_of_nodes():
            print("no more queries to go")
            break
        q = median_node(g, mu, sp_len)
        queried_nodes.add(q)
        
        if debug:
            print('query node: {}'.format(q))
        
        if save_logs:
            centroids.append(q)
        found_source = True
        if np.isinf(infection_times[q]):
            found_source = False
            if inf_reward_table:
                reward = {n: inf_reward_table[(n, q)] for n in g.nodes_iter()}
            else:
                reward = reward_by_uninfected_node(g, q)
        else:
            for u in g.neighbors(q):
                if infection_times[u] < infection_times[q]:
                    queried_nodes.add(u)
                    if direction_reward_table:
                        reward = {n: direction_reward_table[(n, u, q)] for n in g.nodes_iter()}
                    else:
                        reward = reward_by_infection_direction(g, q, u)                    
                    found_source = False
                    break
        if found_source:
            assert source == q
            break
            
        for n in g.nodes_iter():
            mu[n] *= reward[n]
        mu = normalize_mu(mu)
    return len(queried_nodes - obs_nodes)

In [ ]:
dir_tbl, inf_tbl = build_reward_table(g)

In [ ]:
counts = experiment_mwu_multiple_rounds(g, dir_tbl, inf_tbl)

In [ ]:
g.number_of_nodes()

In [ ]:
from experiment_utils import counts_to_stat
counts_to_stat(counts)